## Stock Price Prediction using GRU

<b>We attempt to predict the future stock behaviour (Predicting the closing price of Google Stocks).

Import libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, GRU, Dropout
from keras.models import Sequential

Loading the Data

In [3]:
data = pd.read_csv("stock price_google.csv")
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,50.050049,52.082081,48.028027,50.220219,50.220219,44659096
1,2004-08-20,50.555557,54.594597,50.300301,54.209209,54.209209,22834343
2,2004-08-23,55.430431,56.796799,54.579578,54.754753,54.754753,18256126
3,2004-08-24,55.675674,55.855858,51.836838,52.487488,52.487488,15247337
4,2004-08-25,52.532532,54.054054,51.991993,53.053055,53.053055,9188602


In [4]:
data = data.drop('Adj Close', axis=1)
data.head()

,Date,Open,High,Low,Close,Volume
0,2004-08-19,50.050049,52.082081,48.028027,50.220219,44659096
1,2004-08-20,50.555557,54.594597,50.300301,54.209209,22834343
2,2004-08-23,55.430431,56.796799,54.579578,54.754753,18256126
3,2004-08-24,55.675674,55.855858,51.836838,52.487488,15247337
4,2004-08-25,52.532532,54.054054,51.991993,53.053055,9188602


Normalizing the Closing price of the stocks on the time periods.

In [5]:
data = data.sort_values('Date')
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

In [6]:
scaled_data[:10]

array([[5.60505519e-05],
       [1.40975800e-03],
       [1.59489433e-03],
       [8.25473121e-04],
       [1.01740448e-03],
       [1.34181828e-03],
       [1.04288132e-03],
       [3.39701332e-04],
       [4.00847389e-04],
       [4.07643771e-05]])

In [7]:
training_data_len = int(len(scaled_data)*0.8)
train_data = scaled_data[0:training_data_len, :]
train_data[:10]

array([[5.60505519e-05],
       [1.40975800e-03],
       [1.59489433e-03],
       [8.25473121e-04],
       [1.01740448e-03],
       [1.34181828e-03],
       [1.04288132e-03],
       [3.39701332e-04],
       [4.00847389e-04],
       [4.07643771e-05]])

In [8]:
x_train = []
y_train = []
for i in range(60, len(train_data)): #for creating sequences of 60 data points (common approach in sequence modeling)
    x_train.append(train_data[i-60:i, 0]) #here 0 is to select the first (and only) feature in the sequence
    y_train.append(train_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [9]:
test_data = scaled_data[training_data_len-60:, :]
x_test = []
y_test = data['Close'][training_data_len:].values
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

Defining the model

In [10]:
x_train.shape

(3484, 60, 1)

In [11]:
model = Sequential()
model.add(GRU(units=50, return_sequences=True, input_shape = (x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(GRU(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 60, 50)            7950      
                                                                 
 dropout (Dropout)           (None, 60, 50)            0         
                                                                 
 gru_1 (GRU)                 (None, 60, 50)            15300     
                                                                 
 dropout_1 (Dropout)         (None, 60, 50)            0         
                                                                 
 gru_2 (GRU)                 (None, 50)                15300     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 

Evaluation of Model

In [12]:
# compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=50, batch_size=32)
scores = model.evaluate(x_test, y_test)
print(f'Test Loss: {scores}')


Epoch 1/50

109/109 [==============================] - 31s 113ms/step - loss: 0.0015
Epoch 2/50
109/109 [==============================] - 14s 132ms/step - loss: 2.9745e-04
Epoch 3/50
109/109 [==============================] - 14s 129ms/step - loss: 2.4673e-04
Epoch 4/50
109/109 [==============================] - 13s 119ms/step - loss: 2.3614e-04
Epoch 5/50
109/109 [==============================] - 13s 122ms/step - loss: 1.8938e-04
Epoch 6/50
109/109 [==============================] - 14s 132ms/step - loss: 1.9192e-04
Epoch 7/50
109/109 [==============================] - 13s 120ms/step - loss: 1.6646e-04
Epoch 8/50
109/109 [==============================] - 12s 115ms/step - loss: 1.7072e-04
Epoch 9/50
109/109 [==============================] - 12s 107ms/step - loss: 1.8603e-04
Epoch 10/50
109/109 [==============================] - 12s 110ms/step - loss: 1.7789e-04
Epoch 11/50
109/109 [==============================] - 12s 109ms/step - loss: 1.5313e-04
Epoch 12/50
109/109 [===========

In [13]:
# make predictions
pred = model.predict(x_test)
pred = scaler.inverse_transform(pred)

28/28 [==============================] - 4s 37ms/step


In [14]:
pred

array([[1183.21   ],
       [1176.7787 ],
       [1173.9054 ],
       [1174.1917 ],
       [1178.6855 ],
       [1176.432  ],
       [1175.9883 ],
       [1179.8341 ],
       [1182.5085 ],
       [1187.3271 ],
       [1190.5872 ],
       [1192.8877 ],
       [1194.2422 ],
       [1196.2228 ],
       [1188.738  ],
       [1180.766  ],
       [1172.9625 ],
       [1165.02   ],
       [1145.6147 ],
       [1131.4066 ],
       [1130.6876 ],
       [1125.6005 ],
       [1128.7413 ],
       [1129.6335 ],
       [1121.2673 ],
       [1116.8467 ],
       [1116.1619 ],
       [1116.4247 ],
       [1100.6686 ],
       [1101.3301 ],
       [1098.2646 ],
       [1081.5259 ],
       [1073.425  ],
       [1080.2723 ],
       [1083.6372 ],
       [1080.3394 ],
       [1073.2845 ],
       [1072.3135 ],
       [1082.5378 ],
       [1085.7369 ],
       [1081.7065 ],
       [1071.3634 ],
       [1064.1724 ],
       [1061.7908 ],
       [1064.6729 ],
       [1065.6166 ],
       [1054.417  ],
       [1046.